## 05_Modeling.ipynb
###  **🎯 Objective:**

The objective of this notebook is to define, compile, and train a Convolutional Neural Network (CNN) model to detect powdery mildew in cherry leaf images. This phase follows the preprocessing stage and forms the core of the modeling step in the CRISP-DM methodology.



## Model Architecture
We define a deep learning model using TensorFlow Keras. The model consists of:
 - Convolutional layers for feature extraction
 - MaxPooling layers to reduce spatial dimensions
 - GlobalAveragePooling to reduce the feature map
 - Dense layers for decision making
 - Dropout layer for regularization
 - Sigmoid activation for binary classification

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import json, pickle
from tensorflow.keras.optimizers.legacy import Adam

2025-06-05 03:42:14.534203: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# === Model Definition ===
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    # Dense(1, activation='sigmoid')
    Dense(2, activation='softmax') 
])

### Compile the Model
We compile the model using:
 - Loss function: binary_crossentropy (for binary classification)
 - Optimizer: adam
 - Evaluation metric: accuracy

In [3]:
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

 ### Load Train and Validation Data

### Load Preprocessed Data
Using ImageDataGenerator, we load:
 - Augmented training images
 - Rescaled validation images

This step prepares the model to be trained on realistic and varied data, improving generalization.

In [4]:
# === Image Augmentation ===
img_shape = (256, 256)
batch_size = 32

train_aug = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.10,
    height_shift_range=0.10,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    rescale=1./255
)

test_aug = ImageDataGenerator(rescale=1./255)

In [5]:
# === Data Loaders with class_mode='categorical' ===
train_data = train_aug.flow_from_directory(
    "../inputs/split-leaves/train",
    target_size=img_shape,
    batch_size=batch_size,
    class_mode='categorical'  # <-- Updated from 'binary'
)

val_data = test_aug.flow_from_directory(
    "../inputs/split-leaves/validation",
    target_size=img_shape,
    batch_size=batch_size,
    class_mode='categorical',  # <-- Updated from 'binary'
    shuffle=False
)


Found 3820 images belonging to 2 classes.
Found 1511 images belonging to 2 classes.


In [6]:
# === Summary & Training ===
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 128)       0

### Train the Model and Save Artifacts

### Training the Model
We train the model using model.fit(...) with:
 - EarlyStopping to prevent overfitting
 - ModelCheckpoint to save the best-performing model based on validation loss
 - We run the model for up to 20 epochs, monitoring both training and validation accuracy/loss.

In [7]:
import tensorflow as tf
import keras
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

TensorFlow version: 2.13.0
Keras version: 2.13.1


In [8]:
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("../outputs/mildew_model_softmax.h5", save_best_only=True)
]

history = model.fit(
    train_data,
    epochs=20,
    validation_data=val_data,
    callbacks=callbacks
)

Epoch 1/20
120/120 [==============================] - 158s 1s/step - loss: 0.4711 - accuracy: 0.7613 - val_loss: 0.0852 - val_accuracy: 0.9828
Epoch 2/20


/Users/dappiah/Documents/projects/SmartLeaf/venv/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


120/120 [==============================] - 159s 1s/step - loss: 0.0723 - accuracy: 0.9804 - val_loss: 0.1364 - val_accuracy: 0.9729
Epoch 3/20
120/120 [==============================] - 168s 1s/step - loss: 0.0422 - accuracy: 0.9853 - val_loss: 0.0476 - val_accuracy: 0.9894
Epoch 4/20
120/120 [==============================] - 169s 1s/step - loss: 0.0247 - accuracy: 0.9927 - val_loss: 0.0272 - val_accuracy: 0.9927
Epoch 5/20
120/120 [==============================] - 159s 1s/step - loss: 0.0585 - accuracy: 0.9809 - val_loss: 0.0507 - val_accuracy: 0.9868
Epoch 6/20
120/120 [==============================] - 158s 1s/step - loss: 0.0314 - accuracy: 0.9924 - val_loss: 0.0690 - val_accuracy: 0.9841
Epoch 7/20
120/120 [==============================] - 155s 1s/step - loss: 0.0112 - accuracy: 0.9979 - val_loss: 0.0046 - val_accuracy: 0.9980
Epoch 8/20
120/120 [==============================] - 154s 1s/step - loss: 0.0213 - accuracy: 0.9919 - val_loss: 0.0143 - val_accuracy: 0.9960
Epoch 9/20

In [9]:
# Save full model
model.save("../outputs/mildew_model.keras")

# Also save JSON + weights
model_json = model.to_json()
with open("../outputs/mildew_model_architecture.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("../outputs/mildew_model_weights.h5")


### Save Model and History
We save the trained model to outputs/mildew_model.h5 and store the training history in:
 - training_history.pkl (binary format)
 - history.json (readable format)

This allows us to later visualize training progress without re-training the model.

In [10]:
# Save training history
import json, pickle

# with open("../outputs/training_history.pkl", "wb") as f:
with open("../outputs/training_history_softmax.pkl", "wb") as f:
    pickle.dump(history.history, f)

# with open("../outputs/history.json", "w") as f:
with open("../outputs/history_softmax.json", "w") as f:
    json.dump(history.history, f)


#### To avoid retraining every time, comment out model.fit(...) and just use this:

In [11]:
# with open("../outputs/training_history.pkl", "rb") as f:
#    history_data = pickle.load(f)

In [12]:
pip freeze | grep tensorflow

tensorflow-cpu==2.13.0
tensorflow-estimator==2.13.0
tensorflow-io-gcs-filesystem==0.37.1
Note: you may need to restart the kernel to use updated packages.
